# Model Building

In [104]:
# Import libraries
import pandas as pd
import boto3
import sagemaker
from pyathena import connect
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from time import gmtime, strftime, sleep
from sagemaker.model import Model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
# View created feature groups
!aws sagemaker list-feature-groups

{
    "FeatureGroupSummaries": [
        {
            "FeatureGroupName": "foodlens-products-feature-group-30-14-49-50",
            "FeatureGroupArn": "arn:aws:sagemaker:us-east-1:654654380268:feature-group/foodlens-products-feature-group-30-14-49-50",
            "CreationTime": "2025-09-30T14:49:51.125000+00:00",
            "FeatureGroupStatus": "Created",
            "OfflineStoreStatus": {
                "Status": "Active"
            }
        }
    ]
}


In [3]:
# Change this to your feature group (ex. "foodlens-products-feature-group-30-14-49-50")
# feature_group_name = None
feature_group_name = "foodlens-products-feature-group-30-14-49-50"
assert feature_group_name is not None

In [55]:
# Initialize SageMaker session and Feature Group
# Setup boto and sagemaker session
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
feature_group = FeatureGroup(name=feature_group_name, sagemaker_session=sess)
prefix = "nutriscore-prediction-xgboost"

# Initialize athena query staging
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

# Create connection to Athena database
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [5]:
# Get the name of the unique identifier column
record_id_name = feature_group.describe()['RecordIdentifierFeatureName']

# Get the Athena table name
query = feature_group.athena_query()
table_name = query.table_name

print(f"Athena Table Name: {table_name}")
print(f"Record Identifier: {record_id_name}")

Athena Table Name: foodlens_products_feature_group_30_14_49_50_1759243791
Record Identifier: code


In [6]:
# Verify 100k unique records in feature store
statement = """SELECT COUNT(DISTINCT "code") 
FROM sagemaker_featurestore.{}""".format(table_name)
print(statement)
df = pd.read_sql(statement, conn)
df

SELECT COUNT(DISTINCT "code") 
FROM sagemaker_featurestore.foodlens_products_feature_group_30_14_49_50_1759243791


/tmp/ipykernel_338/3127886201.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,_col0
0,100000


## Prepare Dataset Splits

In [36]:
# Split data into training/testing/validation/production sets

# Base query template
# We use xxhash64 for its speed. It creates a deterministic hash for each record ID.
query_template = f"""
SELECT *
FROM "{table_name}"
WHERE MOD(ABS(from_big_endian_64(xxhash64(CAST("{record_id_name}" AS VARBINARY)))), 100)
"""

# --- Training Data (40%) ---
train_query_string = query_template + " BETWEEN 0 AND 39"
train_query = feature_group.athena_query()
train_query.run(query_string=train_query_string, output_location=f'{s3_staging_dir}/feature_store_queries/train/')
train_query.wait()
train_df = train_query.as_dataframe(dtype={0: str}) # code col as str

# --- Production Data (40%) ---
prod_query_string = query_template + " BETWEEN 40 AND 79"
prod_query = feature_group.athena_query()
prod_query.run(query_string=prod_query_string, output_location=f'{s3_staging_dir}/feature_store_queries/prod/')
prod_query.wait()
prod_df = prod_query.as_dataframe()

# --- Test Data (10%) ---
test_query_string = query_template + " BETWEEN 80 AND 89"
test_query = feature_group.athena_query()
test_query.run(query_string=test_query_string, output_location=f'{s3_staging_dir}/feature_store_queries/test/')
test_query.wait()
test_df = test_query.as_dataframe()

# --- Validation Data (10%) ---
validation_query_string = query_template + " BETWEEN 90 AND 99"
validation_query = feature_group.athena_query()
validation_query.run(query_string=validation_query_string, output_location=f'{s3_staging_dir}/feature_store_queries/validation/')
validation_query.wait()
val_df = validation_query.as_dataframe()

# Verify the shapes of the resulting DataFrames
print(f"Training set shape:   {train_df.shape}")
print(f"Production set shape: {prod_df.shape}")
print(f"Test set shape:       {test_df.shape}")
print(f"Validation set shape: {val_df.shape}")

Training set shape:   (39955, 30)
Production set shape: (39992, 30)
Test set shape:       (9962, 30)
Validation set shape: (10091, 30)


In [37]:
# View training data samples
pd.set_option('display.max_columns', None)
train_df.head(3)

,code,product_name,nova_group,additives_n,ingredients_n,nutriscore_score,energy_100g,salt_100g,carbohydrates_100g,cholesterol_100g,sodium_100g,fiber_100g,fruits_vegetables_legumes_estimate_from_ingredients_100g,sugars_100g,saturated_fat_100g,trans_fat_100g,fat_100g,proteins_100g,fruits_vegetables_nuts_estimate_from_ingredients_100g,energy_kcal_100g,nova_group_100g,nutrition_score_fr_100g,vitamin_a_100g,iron_100g,calcium_100g,vitamin_c_100g,eventtime,write_time,api_invocation_time,is_deleted
0,0075450060065,Almondmilk,4,3,15,7,138.0,0.1875,6.67,0.000,0.075,0.4,0.000000,6.25,0.0,0.0,1.04,0.42,25.000000,33.0,4.0,7.0,0.0,0.00017,0.162,0.0,1.759244e+09,2025-09-30 14:57:53.922,2025-09-30 14:52:55.000,False
1,0033844001964,Brisket rub seasoning,3,0,7,35,0.0,19.1175,58.82,0.000,7.647,0.0,16.053959,58.82,0.0,0.0,0.00,0.00,16.053959,0.0,3.0,35.0,0.0,0.00000,0.118,0.0,1.759244e+09,2025-09-30 14:57:53.922,2025-09-30 14:52:55.000,False
2,0011110033383,Belgian dark chocolate mousse,4,5,59,17,1715.0,0.2500,22.00,0.018,0.100,1.0,0.000000,16.00,8.0,0.0,12.00,2.00,0.703125,410.0,4.0,17.0,0.0,0.00509,0.070,0.0,1.759244e+09,2025-09-30 14:57:53.922,2025-09-30 14:52:55.000,False


## Data Preprocessing

In [42]:
# Feature selection
target = 'nutriscore_score'
cols_to_drop = [
    target,
    'code',
    'product_name',
    'nutrition_score_fr_100g', # data leakage
    'eventtime',
    'write_time',
    'api_invocation_time',
    'is_deleted']

# Set training data
y_train = train_df[target]
X_train = train_df.drop(columns=cols_to_drop)

# Set testing data
y_test = test_df[target]
X_test = test_df.drop(columns=cols_to_drop)

# Set validation data
y_val = val_df[target]
X_val = val_df.drop(columns=cols_to_drop)

# Set production data
y_prod = prod_df[target]
X_prod = prod_df.drop(columns=cols_to_drop)

# View filtered training data
X_train.head()

,nova_group,additives_n,ingredients_n,energy_100g,salt_100g,carbohydrates_100g,cholesterol_100g,sodium_100g,fiber_100g,fruits_vegetables_legumes_estimate_from_ingredients_100g,sugars_100g,saturated_fat_100g,trans_fat_100g,fat_100g,proteins_100g,fruits_vegetables_nuts_estimate_from_ingredients_100g,energy_kcal_100g,nova_group_100g,vitamin_a_100g,iron_100g,calcium_100g,vitamin_c_100g
0,4,3,15,138.0,0.1875,6.67,0.000,0.075,0.4,0.000000,6.25,0.00,0.0,1.04,0.42,25.000000,33.0,4.0,0.000000,0.00017,0.162,0.0000
1,3,0,7,0.0,19.1175,58.82,0.000,7.647,0.0,16.053959,58.82,0.00,0.0,0.00,0.00,16.053959,0.0,3.0,0.000000,0.00000,0.118,0.0000
2,4,5,59,1715.0,0.2500,22.00,0.018,0.100,1.0,0.000000,16.00,8.00,0.0,12.00,2.00,0.703125,410.0,4.0,0.000000,0.00509,0.070,0.0000
3,4,2,12,749.0,1.0725,14.29,0.000,0.429,3.6,77.195335,0.00,0.00,0.0,12.50,3.57,80.059920,179.0,4.0,0.000107,0.00129,0.000,0.0086
4,3,1,12,954.0,1.8675,2.53,0.063,0.747,0.0,10.632500,0.00,6.33,0.0,17.72,15.19,10.632500,228.0,3.0,0.000190,0.00137,0.000,0.0015


In [46]:
# Perform Standardization scaling for regression (FIT ONLY ON TRAINING DATA)

# Define the preprocessing pipeline
preprocessing_pipeline = Pipeline([
    ('scaler', StandardScaler())
])

# Fit the pipeline on the training data
X_train_scaled = preprocessing_pipeline.fit_transform(X_train)
X_test_scaled = preprocessing_pipeline.transform(X_test)
X_val_scaled = preprocessing_pipeline.transform(X_val)
X_prod_scaled = preprocessing_pipeline.transform(X_prod)

# View scaled training data samples
print(X_train_scaled[:3])

[[ 0.58094185  0.12390217 -0.16114354 -0.53261392 -0.03972053 -0.42308753
  -0.03276853 -0.03971217 -0.27267322 -0.55211701 -0.24357289 -0.39748395
  -0.03763313 -0.39477125 -0.28159577  0.11451789 -0.5327726   0.58094185
  -0.00593822 -0.02843393  0.23544068 -0.07854952]
 [-0.45293333 -0.7667445  -0.62332597 -0.5900532   0.32759595  0.22150571
  -0.03276853  0.32749324 -0.31388262  0.00277803  0.72011327 -0.39748395
  -0.03763313 -0.42213335 -0.29564197 -0.16212306 -0.59023631 -0.45293333
  -0.00593822 -0.03090552  0.11917277 -0.07854952]
 [ 0.58094185  0.71766661  2.38085982  0.12377555 -0.03850779 -0.23360307
  -0.01598734 -0.03849979 -0.21085913 -0.55211701 -0.06484092  0.179566
  -0.03763313 -0.10641681 -0.22875532 -0.63682124  0.12370676  0.58094185
  -0.00593822  0.04309689 -0.00766495 -0.07854952]]


In [48]:
# Prepare data splits for upload
# Convert scaled splits back into dataframe.
# We also need to set the target as the first column.
# This is required by the SageMaker XGBoost algorithm.

X_train_df = pd.DataFrame(X_train_scaled)
train_upload_df = pd.concat([pd.DataFrame(y_train).reset_index(drop=True), X_train_df], axis=1)

X_test_df = pd.DataFrame(X_test_scaled)
test_upload_df = pd.concat([pd.DataFrame(y_test).reset_index(drop=True), X_test_df], axis=1)

X_val_df = pd.DataFrame(X_val_scaled)
val_upload_df = pd.concat([pd.DataFrame(y_val).reset_index(drop=True), X_val_df], axis=1)

X_prod_df = pd.DataFrame(X_prod_scaled)
prod_upload_df = pd.concat([pd.DataFrame(y_prod).reset_index(drop=True), X_prod_df], axis=1)

# View scaled training data samples
train_upload_df.head()

,nutriscore_score,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21
0,7,0.580942,0.123902,-0.161144,-0.532614,-0.039721,-0.423088,-0.032769,-0.039712,-0.272673,-0.552117,-0.243573,-0.397484,-0.037633,-0.394771,-0.281596,0.114518,-0.532773,0.580942,-0.005938,-0.028434,0.235441,-0.078550
1,35,-0.452933,-0.766744,-0.623326,-0.590053,0.327596,0.221506,-0.032769,0.327493,-0.313883,0.002778,0.720113,-0.397484,-0.037633,-0.422133,-0.295642,-0.162123,-0.590236,-0.452933,-0.005938,-0.030906,0.119173,-0.078550
2,17,0.580942,0.717667,2.380860,0.123776,-0.038508,-0.233603,-0.015987,-0.038500,-0.210859,-0.552117,-0.064841,0.179566,-0.037633,-0.106417,-0.228755,-0.636821,0.123707,0.580942,-0.005938,0.043097,-0.007665,-0.078550
3,3,0.580942,-0.172980,-0.334462,-0.278299,-0.022548,-0.328902,-0.032769,-0.022545,0.057002,2.116091,-0.358145,-0.397484,-0.037633,-0.093262,-0.176249,1.817151,-0.278539,0.580942,-0.005933,-0.012150,-0.192637,0.066962
4,17,-0.452933,-0.469862,-0.334462,-0.192973,-0.007122,-0.474259,0.025966,-0.007123,-0.313883,-0.184611,-0.358145,0.059107,-0.037633,0.044075,0.212362,-0.329772,-0.193214,-0.452933,-0.005930,-0.010987,-0.192637,-0.053170


In [100]:
# Upload data files to S3
train_file = "train_scaled.csv"
train_upload_df.to_csv(train_file, index=False, header=False)
train_s3_path = sess.upload_data(train_file, key_prefix="{}/train".format(prefix))
print(f"Train data uploaded to: {train_s3_path}")

test_file = "test_scaled.csv"
test_upload_df.to_csv(test_file, index=False, header=False)
test_s3_path = sess.upload_data(test_file, key_prefix="{}/test".format(prefix))
print(f"Test data uploaded to: {test_s3_path}")

# Upload test file without target col for batch transform later
test_features_only_file = "test_features_only.csv"
test_features_only_df = test_upload_df.drop(columns=[target])
test_features_only_df.to_csv(test_features_only_file, index=False, header=False)
test_features_only_s3_path = sess.upload_data(test_features_only_file, key_prefix="{}/test".format(prefix))
print(f"Test data for batch transform uploaded to: {test_features_only_s3_path}")

val_file = "val_scaled.csv"
val_upload_df.to_csv(val_file, index=False, header=False)
val_s3_path = sess.upload_data(val_file, key_prefix="{}/validation".format(prefix))
print(f"Val data uploaded to: {val_s3_path}")

prod_file = "prod_scaled.csv"
prod_upload_df.to_csv(prod_file, index=False, header=False)
prod_s3_path = sess.upload_data(prod_file, key_prefix="{}/prod".format(prefix))
print(f"Prod data uploaded to: {prod_s3_path}")

Train data uploaded to: s3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/train/train_scaled.csv
Test data uploaded to: s3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/test/test_scaled.csv
Test data for batch transform uploaded to: s3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/test/test_features_only.csv
Val data uploaded to: s3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/validation/val_scaled.csv
Prod data uploaded to: s3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/prod/prod_scaled.csv


## Baseline: Training Job and Model Creation

In [ ]:
# insert code here for baseline model (LinearRegression)
# use scaled training data: X_train_df, y_train
# make predictions on scaled test set: X_test_df, y_test
# calculate performance (RMSE, MAE, R-squared)

## XGBoost: Training Job and Model Creation

In [66]:
# Create training job
job_name = "nutrition-score-xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = f"s3://{bucket}/{prefix}/output/{job_name}"

# Specify model
image = sagemaker.image_uris.retrieve(framework="xgboost", region=region, version="1.7-1")
instance_type = 'ml.m5.xlarge'
model_name = job_name

# Set estimator
sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type=instance_type,
    output_path=output_location,
    sagemaker_session=sess,
)

# Set hyperparameters for regression
sm_estimator.set_hyperparameters(
    objective="reg:squarederror",
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    num_round=100,
)

# Set input locations for train and val sets
train_data = sagemaker.inputs.TrainingInput(
    f"s3://{bucket}/{prefix}/train", content_type="text/csv"
)
val_data = sagemaker.inputs.TrainingInput(
    f"s3://{bucket}/{prefix}/validation", content_type="text/csv"
)
data_channels = {"train": train_data, "validation": val_data}

# Launch training job
sm_estimator.fit(inputs=data_channels, job_name=job_name, logs=True)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: nutrition-score-xgb-2025-10-09-00-49-00


2025-10-09 00:49:00 Starting - Starting the training job...
2025-10-09 00:49:33 Starting - Preparing the instances for training...
2025-10-09 00:49:53 Downloading - Downloading input data...
2025-10-09 00:50:18 Downloading - Downloading the training image......
2025-10-09 00:51:30 Training - Training image download completed. Training in progress.
2025-10-09 00:51:30 Uploading - Uploading generated training model/miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-10-09 00:51:20.052 ip-10-2-103-238.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-10-09 00:51:20.126 ip-10-2-103-238.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-10-09:0

## Batch Transform Job for Test Predictions

In [101]:
# Create transformer object from trained estimator
transformer = sm_estimator.transformer(
    instance_count=1,
    instance_type=instance_type
)

# Start the batch transform job
transformer.transform(
    data=test_features_only_s3_path,
    content_type='text/csv'
)

# Wait for the job to complete
print("Batch Transform job started. Waiting for completion...")
transformer.wait()
print("Job complete.")

INFO:sagemaker:Creating model with name: sagemaker-xgboost-2025-10-09-02-01-33-796
INFO:sagemaker:Creating transform job with name: sagemaker-xgboost-2025-10-09-02-01-34-510


............................./miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
[2025-10-09:02:06:25:INFO] No GPUs detected (normal if no gpus installed)
[2025-10-09:02:06:25:INFO] No GPUs detected (normal if no gpus installed)
[2025-10-09:02:06:25:INFO] nginx config: 
worker_processes auto;
daemon off;
pid /tmp/nginx.pid;
error_log  /dev/stderr;
worker_rlimit_nofile 4096;
events {
  worker_connections 2048;
}
http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /dev/stdout combined;
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }
  server {
    listen 8080 deferred;
    client_max_body_size 0;
    keepalive_timeout 3;
    location ~ ^/(ping|invoc

## Compare Model Performance

In [111]:
# Calculate performance on test set

# S3 location of the prediction results
output_s3_path = transformer.output_path

# Download the prediction results into a pandas DataFrame
predictions_df = pd.read_csv(f"{output_s3_path}/{test_features_only_file}.out", header=None)
predictions_df

,0
0,1.043224
1,-0.562068
2,18.242599
3,21.298990
4,18.299925
...,...
9957,3.661992
9958,14.841149
9959,32.303715
9960,8.007389


In [112]:
# Calculate performance metrics
predictions = predictions_df[0]
rmse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"XGBoost Model Performance:")
print(f"RMSE: {rmse}")
print(f"MAE:  {mae}")
print(f"R²:   {r2}")

XGBoost Model Performance:
RMSE: 3.327843795615383
MAE:  1.0611442705088197
R²:   0.9692863623696583


In [ ]:
# Compare baseline RMSE to XGBoost RMSE
# print(f"Baseline RMSE: {baseline_rmse}")
# print(f"XGBoost RMSE: {rmse}")

Some comparison discussion on test results... 

## Upload Trained XGBoost Model to SageMaker Model Registry

In [67]:
# Initialize the sagemaker client
sagemaker_client = boto3.client("sagemaker")

# Get the S3 path to the trained model artifacts
info = sagemaker_client.describe_training_job(TrainingJobName=job_name)
model_data = info["ModelArtifacts"]["S3ModelArtifacts"]
print(f"Model artifacts are located at: {model_data}")

# Define the container image for inference (we defined 'image' earlier)
primary_container = {"Image": image, "ModelDataUrl": model_data}

# Save model to Sagemaker Model Registry
create_model_response = sagemaker_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer=primary_container
)

print(f"\nSuccessfully created model: {create_model_response['ModelArn']}")

Model artifacts are located at: s3://sagemaker-us-east-1-654654380268/nutriscore-prediction-xgboost/output/nutrition-score-xgb-2025-10-09-00-49-00/nutrition-score-xgb-2025-10-09-00-49-00/output/model.tar.gz

Successfully created model: arn:aws:sagemaker:us-east-1:654654380268:model/nutrition-score-xgb-2025-10-09-00-49-00


In [73]:
# Create Endpoint Configuration
endpoint_config_name = f"xgb-nutriscore-config-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
print(f"Creating endpoint configuration: {endpoint_config_name}")

# Create the endpoint configuration
endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": instance_type,
            "InitialInstanceCount": 1
        }
    ]
)
print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

Creating endpoint configuration: xgb-nutriscore-config-2025-10-09-01-06-23
Created EndpointConfig: arn:aws:sagemaker:us-east-1:654654380268:endpoint-config/xgb-nutriscore-config-2025-10-09-01-06-23


In [80]:
# Create real-time endpoint
endpoint_name = f"xgb-nutriscore-endpoint-{strftime('%Y-%m-%d-%H-%M-%S', gmtime())}"
print(f"Creating endpoint: {endpoint_name}")

# Create the endpoint
create_endpoint_response = sagemaker_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

# Wait for endpoint to spin up
sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
print("Waiting for endpoint to be in service...")

while True:
    res = sagemaker_client.describe_endpoint(EndpointName=endpoint_name)
    state = res["EndpointStatus"]
    
    if state == "InService":
        print("Endpoint in Service")
        break
    elif state == "Creating":
        print("Endpoint still creating...")
        sleep(60)
    else:
        print("Endpoint Creation Error - Check Sagemaker Console")
        break

Creating endpoint: xgb-nutriscore-endpoint-2025-10-09-01-12-53
Created Endpoint: arn:aws:sagemaker:us-east-1:654654380268:endpoint/xgb-nutriscore-endpoint-2025-10-09-01-12-53
Waiting for endpoint to be in service...
Endpoint still creating...
Endpoint still creating...
Endpoint still creating...
Endpoint still creating...
Endpoint in Service


## Invoke Real-Time Endpoint Example

In [91]:
# Initialize runtime client
sagemaker_runtime = boto3.client("sagemaker-runtime", region_name=region)

# Prepare test sample
sample_record = X_test.iloc[[0]]
sample_target = y_test[0]

# Scale sample
sample_scaled = preprocessing_pipeline.transform(sample_record)

# Convert scaled sample into csv string
payload = ','.join([str(x) for x in sample_scaled[0]])
print(f"Sending payload: {payload}")
print(f"\nSample Target: {sample_target}")

Sending payload: -2.5206836974260365,-0.766744497897206,-0.8544171815002217,-0.495985968967356,-0.040739237790996925,-0.4437293417266614,-0.02531022224171965,-0.04073056803832349,-0.31388261843214815,-0.5521170054710245,-0.2664872491309747,-0.3073198926339318,-0.03763313191991439,-0.3674091476996953,-0.1842757046919107,-0.6585641731100123,-0.49620478499455123,-2.5206836974260365,-0.005935376041187204,-0.030905520870999983,0.13766993529549926,-0.07854951945892695

Sample Target: 1


In [85]:
# Invoke the Endpoint
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='text/csv',
    Body=payload
)

# Examine the response to see the prediction
prediction = response['Body'].read().decode('utf-8')
print(f"Received prediction (nutriscore_score): {prediction}")

Received prediction (nutriscore_score): 1.0432240962982178



In [92]:
# Delete endpoint to save resources
sagemaker_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'e68437cc-ecb0-4422-bc1a-fe500632b14e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e68437cc-ecb0-4422-bc1a-fe500632b14e',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, no-store, must-revalidate',
   'x-content-type-options': 'nosniff',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 09 Oct 2025 01:29:43 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}